# Pulling Team Statistics for Comparison 

Using Football-API, I'll be pulling data about team statistics to create a Tableau dashboard comparing two teams' season statistics. Since I'm starting on the free plan, only team statistics from the 2021, 2022, and 2023 seasons can be pulled. The purpose for this notebook is using the API to pull data and transform it into a pandas dataframe to be exported for Tableau use (in a xlsx file or csv file to be determined in the future). 

In [2]:
# importing libraries
import requests
import numpy as np
import pandas as pd

To start I'll be pulling team statistics from the Premier League during the 2021-2022 season (aka the 2021 season). Thus during the API requests the parameters to add will be `league=39` (the league id for the English Premier League), `season=2021` (trivial), and `team=___` depending on what the team's id number is (which can be found in the API dashboard).

Teams & Codes for PL 2021-2022 (in order of final standings):
* Man City: 50
* Liverpool: 40
* Chelsea: 49
* Tottenham: 47
* Arsenal: 42
* Man United: 33
* West Ham: 48
* Leicester City: 46
* Brighton: 51
* Wolves: 39
* Newcastle: 34
* Crystal Palace: 52 
* Brentford: 55
* Aston Villa: 66
* Southampton: 41
* Everton: 45
* Leeds United: 63 
* Burnley: 44
* Watford: 38
* Norwich City: 71

In [3]:
# code chunk for the API request
# the documentation for Football-API recommends using this endpoint only once a week for each team
url = "https://v3.football.api-sports.io/teams/statistics?league=39&season=2021&team=50"

payload = {}
headers = {
  'x-rapidapi-key': '01914987bf14c6a4b868a79dfe489f07',
  'x-rapidapi-host': 'v3.football.api-sports.io'
}

response = requests.request("GET", url, headers = headers, data = payload)

print(response.json())

{'get': 'teams/statistics', 'parameters': {'league': '39', 'season': '2021', 'team': '50'}, 'errors': [], 'results': 11, 'paging': {'current': 1, 'total': 1}, 'response': {'league': {'id': 39, 'name': 'Premier League', 'country': 'England', 'logo': 'https://media.api-sports.io/football/leagues/39.png', 'flag': 'https://media.api-sports.io/flags/gb-eng.svg', 'season': 2021}, 'team': {'id': 50, 'name': 'Manchester City', 'logo': 'https://media.api-sports.io/football/teams/50.png'}, 'form': 'LWWWDWDWWLWWWWWWWWWWWWDWWLWWDWDWWWWWDW', 'fixtures': {'played': {'home': 19, 'away': 19, 'total': 38}, 'wins': {'home': 15, 'away': 14, 'total': 29}, 'draws': {'home': 2, 'away': 4, 'total': 6}, 'loses': {'home': 2, 'away': 1, 'total': 3}}, 'goals': {'for': {'total': {'home': 58, 'away': 41, 'total': 99}, 'average': {'home': '3.1', 'away': '2.2', 'total': '2.6'}, 'minute': {'0-15': {'total': 16, 'percentage': '16.67%'}, '16-30': {'total': 13, 'percentage': '13.54%'}, '31-45': {'total': 14, 'percentage

## Processing Data

This section is using Man City's data to experiment and test how to create function to process the rest of the data. Using the nested dictionaries in response's json file, we'll pull the `response` key from the dictionary and transform the necessary information into a pandas dataframe. The variable names for each value will be gross but the objective of this section is creating the process for storing each variable.

In [ ]:
# store response in man_city_response to save data in a different variable not tied
# to the code block being reused for API calls
# since the request was successful (code 200), man_city_response will be the json from 
# the successful the request to reduce navigating the request itself
man_city_response = response.json()

# json response is a bunch of nested dictionaries, so next we'll look at just the response key
# which contains all of the information needed
# NOTE: the results key for the original json response has the number of tabs in the response
# For each of these team responses there should be 11 sections:
# league (dict), team (dict), form (str), fixtures (dict), goals (dict), biggest (dict), 
# clean_sheet (dict), failed_to_score (dict), lineups (list), penalty (dict), and cards (dict)
man_city = man_city_response['response']

# display response portion (man_city variable)
man_city

{'league': {'id': 39,
  'name': 'Premier League',
  'country': 'England',
  'logo': 'https://media.api-sports.io/football/leagues/39.png',
  'flag': 'https://media.api-sports.io/flags/gb-eng.svg',
  'season': 2021},
 'team': {'id': 50,
  'name': 'Manchester City',
  'logo': 'https://media.api-sports.io/football/teams/50.png'},
 'form': 'LWWWDWDWWLWWWWWWWWWWWWDWWLWWDWDWWWWWDW',
 'fixtures': {'played': {'home': 19, 'away': 19, 'total': 38},
  'wins': {'home': 15, 'away': 14, 'total': 29},
  'draws': {'home': 2, 'away': 4, 'total': 6},
  'loses': {'home': 2, 'away': 1, 'total': 3}},
 'goals': {'for': {'total': {'home': 58, 'away': 41, 'total': 99},
   'average': {'home': '3.1', 'away': '2.2', 'total': '2.6'},
   'minute': {'0-15': {'total': 16, 'percentage': '16.67%'},
    '16-30': {'total': 13, 'percentage': '13.54%'},
    '31-45': {'total': 14, 'percentage': '14.58%'},
    '46-60': {'total': 12, 'percentage': '12.50%'},
    '61-75': {'total': 18, 'percentage': '18.75%'},
    '76-90': {'

### Developing Function using Man City information (testing phase)

This section will be a little tedious. It will focus on navigating every tab of the response dictionary and storing each (eventual) column of the resulting dataframe in it's own variable. This will serve as a reference for creating a function to process each response (from other teams) into the same format, a row for the eventual combined dataframe.

#### League Tab

In [19]:
# get league tab
league = man_city['league']

# display league dict
league

{'id': 39,
 'name': 'Premier League',
 'country': 'England',
 'logo': 'https://media.api-sports.io/football/leagues/39.png',
 'flag': 'https://media.api-sports.io/flags/gb-eng.svg',
 'season': 2021}

This league tab should be the same for each team. Thus the only reason we might reference this tab in the future is when we combine data across multiple seasons into one file for Tableau. But until then, there's no variables which need to be stored to add to the final dataframe.

#### Team

In [20]:
# get team tab
team = man_city['team']

# show dict
team

{'id': 50,
 'name': 'Manchester City',
 'logo': 'https://media.api-sports.io/football/teams/50.png'}

Storing the team name and the url for the team's logo will be important for the resulting tableau dashboard. Although the team id number might not be necessary, we'll include it in the dataframe in case it's a useful key for combining tables in the future.

In [21]:
# storing id, name, and logo url
id = team['id'] # int
name = team['name'] # str
logo = team['logo'] # str

#### Form

The form tab (using tab to refer to each key in the original response dictionary) is the only one which isn't a dictionary itself. The form of the team could be a fun feature to include later and storing the value (string) is easy so we'll include it in the dataframe.

In [17]:
# store form 
form = man_city['form'] # str

#### Fixtures 

The fixtures tab include the wins, losses, and ties for the team broken down into home, away, and the total. **IMPORTANT** when developing the function, it will important to check each team plays 19 games at home and 19 games away (the standard for each PL season). Since including the home and away breakdown for the record might be a good addition for team comparisons, we'll include the breakdowns of the records in the dataframe.

In [18]:
# get fixtures tab
fixtures = man_city['fixtures']

# display dict
fixtures

{'played': {'home': 19, 'away': 19, 'total': 38},
 'wins': {'home': 15, 'away': 14, 'total': 29},
 'draws': {'home': 2, 'away': 4, 'total': 6},
 'loses': {'home': 2, 'away': 1, 'total': 3}}

In [26]:
# checking team has 19 home games, 19 away games, and 38 total games
# store values first (so code is neater)
home_games = fixtures['played']['home']
away_games = fixtures['played']['away']
total_games = fixtures['played']['total']

if home_games != 19 | away_games != 19 | total_games != 38:
    raise Exception("Discrepancy in matches played for team")

# if nothing wrong, proceed with storing variables
# wins
home_wins = fixtures['wins']['home']
away_wins = fixtures['wins']['away']
wins = fixtures['wins']['total']
# draws
home_draws = fixtures['draws']['home']
away_draws = fixtures['draws']['away']
draws = fixtures['draws']['total']
# losses
home_losses = fixtures['loses']['home']
away_losses = fixtures['loses']['away']
losses = fixtures['loses']['total']

#### Goals

The goals tab features two main subdirectories, for and against which are also broken down into home, away, and total. Just like the fixture results, storing values for home, away, and total for both subdirectories could be an interesting addition for the dashboard (as well as the averages). Within each subdirectory are two sections for the minute breakdown for the team's goals (0-15, 16-30, etc) and the under_over which is home many games the team score over and under certain thresholds (0.5, 1.5, etc). These would be interesting to include in the dashboard but will be considered later.

In [24]:
# get goals tab
goals = man_city['goals']

# show dict
goals

{'for': {'total': {'home': 58, 'away': 41, 'total': 99},
  'average': {'home': '3.1', 'away': '2.2', 'total': '2.6'},
  'minute': {'0-15': {'total': 16, 'percentage': '16.67%'},
   '16-30': {'total': 13, 'percentage': '13.54%'},
   '31-45': {'total': 14, 'percentage': '14.58%'},
   '46-60': {'total': 12, 'percentage': '12.50%'},
   '61-75': {'total': 18, 'percentage': '18.75%'},
   '76-90': {'total': 17, 'percentage': '17.71%'},
   '91-105': {'total': 6, 'percentage': '6.25%'},
   '106-120': {'total': None, 'percentage': None}},
  'under_over': {'0.5': {'over': 34, 'under': 4},
   '1.5': {'over': 27, 'under': 11},
   '2.5': {'over': 16, 'under': 22},
   '3.5': {'over': 12, 'under': 26},
   '4.5': {'over': 7, 'under': 31}}},
 'against': {'total': {'home': 15, 'away': 11, 'total': 26},
  'average': {'home': '0.8', 'away': '0.6', 'total': '0.7'},
  'minute': {'0-15': {'total': 7, 'percentage': '24.14%'},
   '16-30': {'total': 3, 'percentage': '10.34%'},
   '31-45': {'total': 3, 'percentag

In [37]:
# extract subdirectories for and against
goals_for = goals['for']
goals_against = goals['against']

# show goals_for to see format (subdirectories are organized the same)
goals_for

{'total': {'home': 58, 'away': 41, 'total': 99},
 'average': {'home': '3.1', 'away': '2.2', 'total': '2.6'},
 'minute': {'0-15': {'total': 16, 'percentage': '16.67%'},
  '16-30': {'total': 13, 'percentage': '13.54%'},
  '31-45': {'total': 14, 'percentage': '14.58%'},
  '46-60': {'total': 12, 'percentage': '12.50%'},
  '61-75': {'total': 18, 'percentage': '18.75%'},
  '76-90': {'total': 17, 'percentage': '17.71%'},
  '91-105': {'total': 6, 'percentage': '6.25%'},
  '106-120': {'total': None, 'percentage': None}},
 'under_over': {'0.5': {'over': 34, 'under': 4},
  '1.5': {'over': 27, 'under': 11},
  '2.5': {'over': 16, 'under': 22},
  '3.5': {'over': 12, 'under': 26},
  '4.5': {'over': 7, 'under': 31}}}

In [45]:
# store values
# goals for
# tallies
goals_for_home = goals_for['total']['home']
goals_for_away = goals_for['total']['away']
goals_for_total = goals_for['total']['total']
# averages (have to convert to float since they're stored as strings)
goals_home_avg = float(goals_for['average']['home'])
goals_away_avg = float(goals_for['average']['away'])
goals_avg = float(goals_for['average']['total'])

# goals against
# tallies
goals_against_home = goals_against['total']['home']
goals_against_away = goals_against['total']['away']
goals_against_total = goals_against['total']['total']
# averages (also converted to floats)
against_home_avg = float(goals_against['average']['home'])
against_away_avg = float(goals_against['average']['away'])
against_avg = float(goals_against['average']['total'])


#### Biggest 

This tab features the best statistical performances from the team during the season. These stats might be interesting to include later but for now we'll ignore them. 

#### Clean Sheets

The clean sheets tab is just one dictionary with the home, away, and total clean sheets for the season. Just like fixtures and goals, we'll store each value.

In [40]:
# get clean sheets tab
clean_sheets = man_city['clean_sheet']

# display dictionary
clean_sheets

{'home': 11, 'away': 10, 'total': 21}

In [41]:
# store values
clean_sheets_home = clean_sheets['home']
clean_sheets_away = clean_sheets['away']
clean_sheets_total = clean_sheets['total']

#### Failed to Score

This tab is the opposite of clean sheets, when the team fails to score during a match. While the stat isn't as popular as clean sheets, the tab's structure is identical to clean sheets so storing the values is easy.

In [42]:
# get failed to score tab
failed_scoring = man_city['failed_to_score']

# show dict
failed_scoring

{'home': 2, 'away': 2, 'total': 4}

In [43]:
# store values
failed_home = failed_scoring['home']
failed_away = failed_scoring['away']
failed_total = failed_scoring['total']

#### Penalty

The penalty tab has two subdirectories and "total", the total number of penalties the team got in the season. We'll store the number of total penalties, their conversion record (scored and missed), and the percentages (jic they're useful in the dashboard).

In [44]:
# get penalty tab
penalty = man_city['penalty']

# show dict
penalty

{'scored': {'total': 7, 'percentage': '100.00%'},
 'missed': {'total': 0, 'percentage': '0%'},
 'total': 7}

In [49]:
# store values
# totals
total_pens = penalty['total']
pens_scored = penalty['scored']['total']
pens_missed = penalty['missed']['total']
# conversion (have to get rid of the percentage symbol and convert to float)
pens_converstion = float(penalty['scored']['percentage'][:-1])

#### Lineups 

The lineups tab is actually a list of dictionaries. Each dict in the list contains a formation and the number of games the team used this formation. This stat will be another saved for later because each team will have a different number of lineups used so handling this tab will require additional work.

#### Cards

This tab contains the number of yellow and red cards the team received during the season. However, each subdirectory (red and yellow) doesn't contain a value for the total but rather has a breakdown of how many cards were given during each 15 minute interval of the game. Thus to tally the total of each, we will iterate through the intervals and sum up the totals in those dictionaries. 

In [52]:
# get cards tab
cards = man_city['cards']

# show dict
cards

{'yellow': {'0-15': {'total': 1, 'percentage': '2.38%'},
  '16-30': {'total': 7, 'percentage': '16.67%'},
  '31-45': {'total': 7, 'percentage': '16.67%'},
  '46-60': {'total': 9, 'percentage': '21.43%'},
  '61-75': {'total': 5, 'percentage': '11.90%'},
  '76-90': {'total': 6, 'percentage': '14.29%'},
  '91-105': {'total': 7, 'percentage': '16.67%'},
  '106-120': {'total': None, 'percentage': None}},
 'red': {'0-15': {'total': None, 'percentage': None},
  '16-30': {'total': None, 'percentage': None},
  '31-45': {'total': None, 'percentage': None},
  '46-60': {'total': 1, 'percentage': '100.00%'},
  '61-75': {'total': None, 'percentage': None},
  '76-90': {'total': None, 'percentage': None},
  '91-105': {'total': None, 'percentage': None},
  '106-120': {'total': None, 'percentage': None}}}

In [68]:
# split into yellow and red subdirectories
yellow_cards = cards['yellow']
red_cards = cards['red']

# show one (structure is identical)
yellow_cards

{'0-15': {'total': 1, 'percentage': '2.38%'},
 '16-30': {'total': 7, 'percentage': '16.67%'},
 '31-45': {'total': 7, 'percentage': '16.67%'},
 '46-60': {'total': 9, 'percentage': '21.43%'},
 '61-75': {'total': 5, 'percentage': '11.90%'},
 '76-90': {'total': 6, 'percentage': '14.29%'},
 '91-105': {'total': 7, 'percentage': '16.67%'},
 '106-120': {'total': None, 'percentage': None}}

In [65]:
# iterate through subdirectories to tally total
# yellow cards
# initialize total
yc_total = 0
for interval in yellow_cards.values():
    # check if total is NoneType first
    if interval['total'] is None:
        yc_total += 0
        continue
    # if numeric, can just add int to total
    yc_total += interval['total']

# red cards
rc_total = 0
for interval in red_cards.values():
    # check if total is NoneType first
    if interval['total'] is None:
        rc_total += 0
        continue
    # if numeric, can just add int to total
    rc_total += interval['total']

#### Putting it Together

With all of the values now stored, we can finally combine the information into one row of a dataframe. This will involve putting them into the frame first and then figuring out naming conventions for each of the columns (so the width of the printed frame isn't ridiculous).

In [79]:
# creating dictionary for values before converting to pd.DataFrame
# naming conventions
'''
Team Overview
id: team id number
name: team name
logo: url for team's logo
form : team form (performance in order from left to right)
-----------------------------
Performance
HW : home wins
AW : away wins
TW : total wins (home wins + away wins)
HL : home losses
AL : away losses 
TL : total losses
HD : home draws
AD : away draws
TD : total draws
-----------------------------
Goal Tallies
GFH : goals for at home
GFA : goals for away
GFT : goals for total
GAH : goals against at home
GAA : goals against away
GAT : goals against total
-----------------------------
Goal Averages 
GFH_avg : goals for at home (avg)
GFA_avg : goals for away (avg)
GFT_avg : goals for total (avg)
GAH_avg : goals against at home (avg)
GAA_avg : goals against away (avg)
GAT_avg : goals against total (avg)
-----------------------------
Clean Sheets & Failed Scored
CSH : home clean sheets
CSA : away clean sheets
CST : total clean sheets
FSH : failed to score games at home
FSA : failed to score games away
FST : total games failed to score
-----------------------------
Penalties
PS : penalties scored
PM : penalties missed
PT : total penalties
PC : penalty conversion rate
-----------------------------
Cards
YC : total yellow cards
RC : total red cards
'''

# create dictionary
# include team id columns later
data = {
    'id' : id, 'name' : name, 'logo' : logo,
    'form' : form,
    'HW' : home_wins, 'AW' : away_wins, 'TW' : wins,
    'HL' : home_losses, 'AL' : away_losses, 'TL' : losses,
    'HD' : home_draws, 'AD' : away_draws, 'TD' : draws,
    'GFH' : goals_for_home, 'GFA' : goals_for_away, 'GFT' : goals_for_total,
    'GAH' : goals_against_home, 'GAA' : goals_against_away, 'GAT' : goals_against_total,
    'GFH_avg' : goals_home_avg, 'GFA_avg' : goals_away_avg, 'GFT_avg' : goals_avg,
    'GAH_avg' : against_home_avg, 'GAA_avg' : against_away_avg, "GAT_avg" : against_avg,
    'CSH' : clean_sheets_home, 'CSA' : clean_sheets_away, 'CST' : clean_sheets_total,
    'FSH' : failed_home, 'FSA' : failed_away, 'FST' : failed_total,
    'PS' : pens_scored, 'PM' : pens_missed, 'PT' : total_pens, 'PC' : pens_converstion,
    'YC' : yc_total, 'RC' : rc_total
}

# create pd.DataFrame
data_df = pd.DataFrame(data, index = [0])

# show dataframe
data_df

,id,name,logo,form,HW,AW,TW,HL,AL,TL,...,CST,FSH,FSA,FST,PS,PM,PT,PC,YC,RC
0,50,Manchester City,https://media.api-sports.io/football/teams/50.png,LWWWDWDWWLWWWWWWWWWWWWDWWLWWDWDWWWWWDW,15,14,29,2,1,3,...,21,2,2,4,7,0,7,100.0,42,1
